In [18]:
from tensorflow.keras.models import Model
from tensorflow.keras import models
from tensorflow.keras.utils import plot_model
from tensorflow.keras import callbacks
from tensorflow.keras.layers import Input,LSTM,Dense
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import pickle


In [44]:

input_texts_translation=[]
target_texts_translation=[]
input_characters_translation=set()
target_characters_translation=set()

In [45]:

with open('word.txt','r',encoding='utf-8') as f:
    rows=f.read().split('\n')
    count=0
for row in rows[:88320]:
  
    input_text,target_text = row.split('\t')
    
    target_text='\t' + target_text + '\n'
    input_texts_translation.append(input_text.lower())
    target_texts_translation.append(target_text.lower())
   
    input_characters_translation.update(list(input_text.lower()))
    target_characters_translation.update(list(target_text.lower()))
    

In [46]:
len(input_texts_translation)

88320

In [47]:

input_characters_translation = sorted(list(input_characters_translation))
target_characters_translation = sorted(list(target_characters_translation))

num_en_chars_translation = len(input_characters_translation)
num_dec_chars_translation = len(target_characters_translation)

max_input_length_translation = max([len(i) for i in input_texts_translation])
max_target_length_translation = max([len(i) for i in target_texts_translation])


In [48]:
def bagofcharacters_translation(input_texts,target_texts):
  
    en_in_data=[] ; dec_in_data=[] ; dec_tr_data=[]
    pad_en=[1]+[0]*(len(input_characters_translation)-1)
    pad_dec=[0]*(len(target_characters_translation)) ; pad_dec[2]=1
    cv=CountVectorizer(binary=True,tokenizer=lambda txt: txt.split(),stop_words=None,analyzer='char')
     
    for i,(input_t,target_t) in enumerate(zip(input_texts,target_texts)):     
          
        cv_inp= cv.fit(input_characters_translation)
        en_in_data.append(cv_inp.transform(list(input_t)).toarray().tolist())
        cv_tar= cv.fit(target_characters_translation)		
        dec_in_data.append(cv_tar.transform(list(target_t)).toarray().tolist())
        dec_tr_data.append(cv_tar.transform(list(target_t)[1:]).toarray().tolist())
        if len(input_t) < max_input_length_translation:            
            for _ in range(max_input_length_translation-len(input_t)):
                en_in_data[i].append(pad_en)                
        if len(target_t) < max_target_length_translation:
              for _ in range(max_target_length_translation-len(target_t)):
                dec_in_data[i].append(pad_dec)
        if (len(target_t)-1) < max_target_length_translation:
              for _ in range(max_target_length_translation-len(target_t)+1):
                dec_tr_data[i].append(pad_dec)


    en_in_data_translation=np.array(en_in_data,dtype="float32")
    dec_in_data_translation=np.array(dec_in_data,dtype="float32")
    dec_tr_data_translation=np.array(dec_tr_data,dtype="float32")

     
    return en_in_data_translation,dec_in_data_translation,dec_tr_data_translation

In [49]:

en_inputs_translation = Input(shape=(None, num_en_chars_translation))


encoder_translation = LSTM(256, return_state=True)


en_outputs_translation, state_h_translation, state_c_translation = encoder_translation(en_inputs_translation)
en_states_translation = [state_h_translation, state_c_translation]

In [50]:

dec_inputs_translation = Input(shape=(None, num_dec_chars_translation))


dec_lstm_translation = LSTM(256, return_sequences=True, return_state=True)

dec_outputs_translation, _, _ = dec_lstm_translation(dec_inputs_translation, initial_state=en_states_translation)

dec_dense_translation = Dense(num_dec_chars_translation, activation="softmax")
dec_outputs_translation = dec_dense_translation(dec_outputs_translation)

In [51]:
model_translation = Model([en_inputs_translation, dec_inputs_translation], dec_outputs_translation)

In [52]:
pickle.dump({'input_characters':input_characters_translation,'input_texts':input_texts_translation[:92],'target_characters':target_characters_translation, 'max_input_length':max_input_length_translation, 'max_target_length':max_target_length_translation, 'num_en_chars':num_en_chars_translation, 'num_dec_chars':num_dec_chars_translation}, open("training_data_translation.pkl", "wb"))


In [53]:
en_in_data_translation,dec_in_data_translation,dec_tr_data_translation = bagofcharacters_translation(input_texts_translation,target_texts_translation)


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:551: UserWarning: The parameter 'tokenizer' will not be used since 'analyzer' != 'word'
  warnings.warn(


In [54]:




model_translation.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)



In [55]:
earlystopping = callbacks.EarlyStopping(monitor = "val_loss", mode="min" , patience = 5,restore_best_weights=True)

In [56]:
model_translation.fit(
    [en_in_data_translation, dec_in_data_translation],
    dec_tr_data_translation,
    batch_size=128,
    epochs=7,
    validation_split=0.2,
    
)

# Save model
model_translation.save("model_translation")

Epoch 1/7
552/552 [==============================] - 1144s 2s/step - loss: 0.5110 - accuracy: 0.8678 - val_loss: 0.0951 - val_accuracy: 0.9734
Epoch 2/7
552/552 [==============================] - 1479s 3s/step - loss: 0.0358 - accuracy: 0.9878 - val_loss: 0.0123 - val_accuracy: 0.9946
Epoch 3/7
552/552 [==============================] - 895s 2s/step - loss: 0.0137 - accuracy: 0.9941 - val_loss: 0.0267 - val_accuracy: 0.9894
Epoch 4/7
552/552 [==============================] - 997s 2s/step - loss: 0.0099 - accuracy: 0.9953 - val_loss: 0.0080 - val_accuracy: 0.9961
Epoch 5/7
552/552 [==============================] - 957s 2s/step - loss: 0.0086 - accuracy: 0.9956 - val_loss: 0.0072 - val_accuracy: 0.9958
Epoch 6/7
552/552 [==============================] - 1134s 2s/step - loss: 0.0082 - accuracy: 0.9957 - val_loss: 0.0165 - val_accuracy: 0.9940
Epoch 7/7
552/552 [==============================] - 1355s 2s/step - loss: 0.0079 - accuracy: 0.9958 - val_loss: 0.0074 - val_accuracy: 0.9958


INFO:tensorflow:Assets written to: model_translation\assets


INFO:tensorflow:Assets written to: model_translation\assets


In [57]:

datafile_translation = pickle.load(open("training_data_translation.pkl","rb"))
input_characters_translation = datafile_translation['input_characters']
target_characters_translation = datafile_translation['target_characters']
max_input_length_translation = datafile_translation['max_input_length']
max_target_length_translation = datafile_translation['max_target_length']
num_en_chars_translation = datafile_translation['num_en_chars']
num_dec_chars_translation = datafile_translation['num_dec_chars']
input_texts_translation=datafile_translation['input_texts']


In [39]:
target_characters_translation

['\t',
 '\n',
 ' ',
 'ं',
 'आ',
 'उ',
 'क',
 'ख',
 'ग',
 'घ',
 'च',
 'ज',
 'झ',
 'ट',
 'ड',
 'ण',
 'त',
 'द',
 'न',
 'प',
 'फ',
 'ब',
 'भ',
 'म',
 'य',
 'र',
 'ल',
 'ळ',
 'व',
 'श',
 'स',
 'ह',
 'ा',
 'ि',
 'ी',
 'ु',
 'ू',
 'े',
 'ो',
 '्']

In [23]:


model_translation = models.load_model("model_translation")

enc_outputs_translation, state_h_enc_translation, state_c_enc_translation = model_translation.layers[2].output 


en_model_translation = Model(model_translation.input[0], [state_h_enc_translation, state_c_enc_translation])

dec_state_input_h_translation = Input(shape=(256,))
dec_state_input_c_translation = Input(shape=(256,))
dec_states_inputs_translation = [dec_state_input_h_translation, dec_state_input_c_translation]


dec_outputs_translation, state_h_dec_translation, state_c_dec_translation = dec_lstm_translation(
    model_translation.input[1], initial_state=dec_states_inputs_translation
)

dec_states_translation = [state_h_dec_translation, state_c_dec_translation]
dec_dense_translation = model_translation.layers[4]
dec_outputs_translation = dec_dense_translation(dec_outputs_translation)


dec_model_translation = Model(
    [model_translation.input[1]] + dec_states_inputs_translation, [dec_outputs_translation] + dec_states_translation
)

In [24]:
def decode_sequence_translation(input_seq):
    
    reverse_target_char_index = dict(enumerate(target_characters_translation))
    
    states_value = en_model_translation.predict(input_seq)
    cv=CountVectorizer(binary=True,tokenizer=lambda txt: txt.split(),stop_words=None,analyzer='char')
 
    co=cv.fit(target_characters_translation) 
    target_seq=np.array([co.transform(list("\t")).toarray().tolist()],dtype="float32")

   
    stop_condition = False
   
    decoded_sentence = ""

    while not stop_condition:
        
        output_chars, h, c = dec_model_translation.predict([target_seq] + states_value)

        
        char_index = np.argmax(output_chars[0, -1, :])
        text_char = reverse_target_char_index[char_index]
        decoded_sentence += text_char
        
      
        if text_char == "\n" or len(decoded_sentence) > max_target_length_translation:
            stop_condition = True
   
        target_seq = np.zeros((1, 1, num_dec_chars_translation))
        target_seq[0, 0, char_index] = 1.0
        states_value = [h, c]
   
    return decoded_sentence

In [25]:
    def bagofcharacter_translation(input_t):
        cv=CountVectorizer(binary=True,tokenizer=lambda txt:
        txt.split(),stop_words=None,analyzer='char') 
        en_in_data=[] ; pad_en=[1]+[0]*(len(input_characters_translation)-1)
    
        cv_inp= cv.fit(input_characters_translation)
        en_in_data.append(cv_inp.transform(list(input_t)).toarray().tolist())
    
        if len(input_t)< max_input_length_translation:
          for _ in range(max_input_length_translation-len(input_t)):
            en_in_data[0].append(pad_en)
    
        return np.array(en_in_data,dtype="float32")
    

 
        


In [26]:
for x in input_texts_translation[:92]:
    output_texts=[]
    print("input : "+ x)
    #x=  input( 'Enter eng sentence : ' ) 
    input_text = x.split(' ') 
    output_texts=""  
    en_in_data = bagofcharacter_translation( x.lower()+".")    
    x=decode_sequence_translation(en_in_data)
    output_texts+=" "+ x         
    print("output : "+output_texts)
